In [1]:
# libraries
import os
from dotenv import load_dotenv
import pandas as pd
import requests
import time
import psycopg2 as ps

In [2]:
# global keys

load_dotenv()

# API key
API_key = os.getenv("API_key")
# Channel ID for ZHONG.TV
ChannelID = os.getenv("ChannelID")

In [5]:
def search_api_call (pageToken):

    # URl for API
    URL_search = "https://www.googleapis.com/youtube/v3/search?key="+API_key+"&channelId="+ChannelID+"&part=id,snippet&order=date&maxResults=50&pageToken="+pageToken
    
    # GET request
    response_search = requests.get(URL_search).json()

    return response_search

In [7]:
def extract_video_details(df, response_search):

    # extraction of first page with pageToken = ""
    for video in response_search['items']:
        # extract desired values through each iteration for search route
        video_id, video_date, video_title = extract_from_search(video)

        # extract desired values through each iteration for video route
        video_viewCount, video_likeCount, video_commentCount = extract_from_video(video_id) 

        temp_df = pd.DataFrame({ "video_id":[video_id], "video_title":[video_title], "video_viewCount":[video_viewCount], 
                                "video_likeCount":[video_likeCount],"video_commentCount":[video_commentCount], "video_date":[video_date]})

        df = pd.merge(df, temp_df, how = "outer")

    return df

In [9]:
def extract_from_search (video):
    video_id = video['id']['videoId']
    video_date = video['snippet']['publishedAt']
    video_date = video_date.split('T')[0]
    video_title = video['snippet']['title']
    
    return video_id, video_date, video_title

In [11]:
def extract_from_video(video_id):
    URL_videos = "https://www.googleapis.com/youtube/v3/videos?id=" + video_id + "&part=statistics&maxResults=50&key=" + API_key
    response_videos = requests.get(URL_videos).json()
    time.sleep(2)

    # extracting values for each column
    video_viewCount = response_videos['items'][0]['statistics']['viewCount']
    video_likeCount = response_videos['items'][0]['statistics']['likeCount']
    video_commentCount = response_videos['items'][0]['statistics']['commentCount']

    return video_viewCount, video_likeCount, video_commentCount
    

In [ ]:
# main function  

# first API call
response_search = search_api_call(pageToken = "")
time.sleep(1)


df = pd.DataFrame(columns = ["video_id", "video_title", "video_viewCount", "video_likeCount","video_commentCount", "video_date"])
df = extract_video_details(df, response_search)

more_pages =  True

while more_pages:
    # end loop if nextPageTokens are absent
    if not response_search['nextPageToken']:
        more_pages = False
    else:
        try:
            nextPageToken = response_search['nextPageToken']
            response_search = search_api_call(nextPageToken)
            time.sleep(1)

            df = extract_video_details(df, response_search) 
        except:
            break

In [35]:
# function to establish connection
def connect_database(hostName, dbName, port, username, password):
    try:
        connection = ps.connect(host=hostName, database=dbName, user=username, password=password, port=port)
    except ps.OperationalError as Error:
        raise Error
    else:
        print("connected!")

    return connection

In [37]:
def create_oncloudTable(connection):
    # cursor for SQL queries to work on database
    cursor = connection.cursor()
    
    # query for creating table 
    sqlQuery_for_creating_table = ("""CREATE TABLE IF NOT EXISTS videoDB (
                video_id VARCHAR(255) PRIMARY KEY,
                video_title VARCHAR(255),
                video_viewCount INT NOT NULL,
                video_likeCount INT NOT NULL,
                video_commentCount INT NOT NULL,
                video_date DATE NOT NULL
                );""")
    cursor.execute(sqlQuery_for_creating_table)

    # commit connection
    connection.commit()

    # close cursor
    cursor.close()

In [39]:
def update_oncloudTable(cursor, video_id, video_title, video_viewCount, video_likeCount, video_commentCount, video_date):

    # query for updating rows of oncloud db
    sqlQuery_for_updating = ("""UPDATE videoDB
                                SET video_title=%s, video_viewCount=%s, video_likeCount=%s, video_commentCount=%s
                                WHERE video_id=%s;""")

    rows_to_update = (video_title, video_viewCount, video_likeCount, video_commentCount, video_id)
    # execute query and input values for placeholders
    cursor.execute(sqlQuery_for_updating, rows_to_update)

In [41]:
def insertion(connection, video_id, video_title, video_viewCount, video_likeCount, video_commentCount, video_date):

    # cursor for SQL queries to work on database
    cursor = connection.cursor()
    
    # query for inserting rows into  oncloud db
    sqlQuery_for_insertion =  ("""INSERT INTO videoDB (video_id, video_title, video_viewCount, video_likeCount, video_commentCount, video_date) VALUES (%s, %s, %s, %s, %s, %s);""")
    print("inserted")

    try:
        cursor.execute(sqlQuery_for_insertion, (video_id, video_title, video_viewCount, video_likeCount, video_commentCount, video_date))
        connection.commit()
    except Exception as e:
        connection.rollback()

    # close cursor
    cursor.close()

In [43]:
def insert_into_oncloudTable(connection, df_for_insertion):

    for index, row in df_for_insertion.iterrows():
        insertion(connection, row['video_id'], row['video_title'], row['video_viewCount'], row['video_likeCount'], row['video_commentCount'], row['video_date'])


In [45]:
def upload_to_oncloudTable(df, df_for_insertion, connection):
    # cursor for SQL queries to work on database
    cursor = connection.cursor()

    # iterate through rows of db to check if video already exists
    for index, row in df.iterrows():
        # check for existence
        if exists(cursor, row['video_id']):
            update_oncloudTable(cursor, row['video_id'], row['video_title'], row['video_viewCount'], row['video_likeCount'], row['video_commentCount'], row['video_date'])
            print("database updated")
        else:
            # store new values into temp_df
            temp_df = pd.DataFrame({"video_id":[row['video_id']], "video_title":[row['video_title']], "video_viewCount":[row['video_viewCount']], "video_likeCount":[row['video_likeCount']],"video_commentCount":[row['video_commentCount']], "video_date":[row['video_date']]})
            
            # insert these values into df_for_insertion
            df_for_insertion = pd.merge(df_for_insertion, temp_df, how="outer")
        
    # commit connection
    connection.commit()

    # close cursor
    cursor.close()

    return df_for_insertion

In [47]:
def exists(cursor, video_id):
    
    # check for rows returned
    cursor.execute("""SELECT video_id FROM videoDB WHERE video_id = %s;""", (video_id,))
    if cursor.fetchone():
        return True
        
    return False

In [49]:
# credentials of oncloud DB
hostName = os.getenv("hostName")
dbName = os.getenv("dbName")
port = os.getenv("port")
username = os.getenv("username")
password = os.getenv("password")

connection = connect_database(hostName, dbName, port, username, password)

# call on create table function
create_oncloudTable(connection)

# create df_for_insertion
df_for_insertion = pd.DataFrame(columns = ["video_id", "video_title", "video_viewCount", "video_likeCount","video_commentCount", "video_date"])

# call on function to update existing values into oncloud database, and return df_for_insertion that stores new values
df_for_insertion = upload_to_oncloudTable(df, df_for_insertion, connection)

# call on function to insert new values into oncloud db
insert_into_oncloudTable(connection, df_for_insertion)



connected!
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
database updated
dat